In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

file_in = '../data/5-image.png'

img = cv2.imread(file_in)
#enleve 200 pixels a gauche et a droite 
image_resized = img[:, 400:-400]
img_out = np.zeros_like(image_resized) 

# Auto balanced
for channel_index in range(3):
    #calcul histogramme
    hist, bins = np.histogram(image_resized[..., channel_index].ravel(), 256, (0, 256))
    #calcul de la valeur min et max
    bmin = np.min(np.where(hist > (hist.sum() * 0.0005)))
    bmax = np.max(np.where(hist > (hist.sum() * 0.0005)))
    #appliquer la transformation
    img_out[..., channel_index] = np.clip(image_resized[..., channel_index], bmin, bmax)
    #normaliser
    img_out[..., channel_index] = ((img_out[..., channel_index] - bmin) / (bmax - bmin) * 255)

def cvt2bokeh(img):
    h, w = img.shape[:2]
    container = np.empty((h, w), dtype=np.uint32)
    view = container.view(dtype=np.uint8).reshape((h, w, 4))
    view[:, :, 0] = img[::-1, :, 0]  # copy red channelmatch_aspect
    view[:, :, 1] = img[::-1, :, 1]  # copy blue channel
    view[:, :, 2] = img[::-1, :, 2]  # copy green channel
    view[:, :, 3] = 255
    return container

from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource

image = cvt2bokeh(cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB))
image_out2 = cvt2bokeh(cv2.cvtColor(img_out, cv2.COLOR_BGR2RGB))

output_notebook()

source1 = ColumnDataSource(data=dict(image=[image]))
source2 = ColumnDataSource(data=dict(image=[image_out2]))

# Calculate figure dimensions proportional to the image
image_height, image_width = image_resized.shape[:2]

p1 = figure(title="Image originale", 
            x_range=(0, image_width), y_range=(0, image_height),
            width=image_width//2, height=image_height//2)
p2 = figure(title="Image après traitement (auto balance)", 
            x_range=p1.x_range, y_range=p1.y_range,
            width=image_width//2, height=image_height//2)

p1.image_rgba(image='image', source=source1, x=0, y=0, dw=image_width, dh=image_height)
p2.image_rgba(image='image', source=source2, x=0, y=0, dw=image_width, dh=image_height)

grid = gridplot([[p1, p2]])
show(grid)

from PIL import Image

img = Image.open(file_in)
img = np.array(image_resized)
hist, bins = np.histogram(img.flatten(), bins=256, density=True)

p = figure(width=image_width, height=image_height, toolbar_location=None,
           title="Histogramme de l'image avec bokeh")

p.quad(top=hist, bottom=0, left=bins[:-1], right=bins[1:],
       fill_color="skyblue", line_color="white", line_alpha=0.5)

p.xaxis.axis_label = "Valeurs de pixel"
p.yaxis.axis_label = "Fréquence"

show(p)
